# Traffic Crashes in the City of San Francisco

## About the data

**Source:** This data contains public information about the crashes resulting in an injury in the City of San Francisco. The datab was pulled from `TransBASE.sfgov.org` in July 2024. There are several types of report; this data is based on the "Traffic Crashes Resulting in Injury: Parties Involved" found in https://data.sfgov.org/d/8gtc-pjc6/). 

**Preprocessing**: Some columns from the original data were dropped. For those remaining, he content was modified as follows:
- Date formats were changed to %m/%d/%Y %I:%M:%S %p
- "weather_1" and "weather_2" columns: empty data replaced by "Not Stated", when "Fog" or 'Other" strings are found, replace eliminate keep only "Fog" and "Other".
- "party_age" column: negative values were removed.
- "tb_latitude" and "tb_longitude" columns: numbers were rounded to 6 decimals.
- "vehicle_year" column: contained values that were either errors or types.
- "vehicle_age" added as the difference in between the year of the collision and the vehicle year; 0 when the difference is negative.
- Data was only kept for collisions between 2016 and 2023, both included.

The following code contains the exact transformation, with `df` being initialized using Pandas and the data downloaded from the above website.

<code># df initialized with the downloaded data
df['collision_datetime'] = pd.to_datetime(df['collision_datetime'], format='%m/%d/%Y %I:%M:%S %p')
for column in ['weather_1', 'weather_2']:
    df.loc[df[column].isna(), column] = 'Not Stated'
    df.loc[df[column].str.contains('Fog'), column] = 'Fog'
    df.loc[df[column].str.contains('Other'), column] = 'Other'
for column in ['party_age']:
    df.loc[df[column] < 0, column] = None
for column in ['tb_latitude', 'tb_longitude']:
    df[column] = df[column].round(6)
if df.columns.str.contains('vehicle_year').sum() > 0:
    f = (df['vehicle_year'] >= 2024) | ((df['vehicle_year'] >= 224) & (df['vehicle_year'] < 1900))
    df.loc[df[f].index, 'vehicle_year'] = df.loc[df[f].index, 'vehicle_year']%100
    f = df['vehicle_year'] < 24
    df.loc[df[f].index, 'vehicle_year'] = df.loc[df[f].index, 'vehicle_year'] + 2000
    f = (df['vehicle_year'] >= 24) & (df['vehicle_year'] <= 99)
    df.loc[df[f].index, 'vehicle_year'] = df.loc[df[f].index, 'vehicle_year'] + 1900
    f = (df['vehicle_year'] >= 100) & (df['vehicle_year'] <= 199)
    df.loc[df[f].index, 'vehicle_year'] = df.loc[df[f].index, 'vehicle_year'] + 1800    
    f = (df['vehicle_year'] >= 200) & (df['vehicle_year'] < 224)
    df.loc[df[f].index, 'vehicle_year'] = df.loc[df[f].index, 'vehicle_year'] + 1800    
    f = (df['collision_datetime'].dt.year-df['vehicle_year']) < 0
    df.loc[df[f].index, 'vehicle_year'] = df[f]['collision_datetime'].dt.year
df['vehicle_age'] = (df['collision_datetime'].dt.year-df['vehicle_year']).astype(pd.Int64Dtype())
df[(df['collision_datetime'].dt.year >= 2016) & (df['collision_datetime'].dt.year <= 2023)].to_csv('sf_traffic.csv', sep=',')</code>

                                                                                                  
The resulting data is available on GitHub `https://github.com/D-G-D/Python/blob/main/Data/sf_traffic.csv`.

**Data format**: Each row of of the data contains the information of a party involved in an accident. The row contains also information about the accident itself, that is the same for all parties (for example: the road where the accident ocurred). The following columns are available: 

- `unique_id`: a unique identifier for each record.
- `case_id_pkey`: the unique identifier of the crash case.
- `tb_latitude`: latitude of the location where the collision occurred.
- `tb_longitude`: longitude of the location where the collision occurred.
- `collision_datetime`: date and time when the collision took place.
- `officer_id`: an ID of the officer who wrote the crash report.
- `primary_rd`: the road where the collision occurred.
- `secondary_rd`: a secondary reference road that `distance` and `direction` are measured from.
- `distance`: distance in ft from a known reference point (often the nearest milepost or intersection).
- `direction`: direction of `distance`.
- `weather_1`: primary weather condition at the time of the collision.
- `weather_2`: additional weather condition at the time of the collision.
- `collision_severity`: the injury level severity of the crash (highest level of injury in crash)
- `type_of_collision`: the general type of crash as determined by the first injury or damage-causing event.
- `mviw`: motor vehicle involved with; describes what, in conjunction with a motor vehicle in-transport, produced the first injury or damage-causing event, on or off the road.
- `ped_action`: the action just prior to the crash of the first pedestrian injured or otherwise involved.    
- `road_surface`: roadway surface condition at the time of the crash in the traffic lane(s) involved.    
- `road_cond_1`: roadway condition at the time of the crash in the traffic lane(s) involved..
- `road_cond_2`: road condition, if a second description is necessary.
- `lighting`: lighting conditions at the crash location and the time of the crash.
- `control_device`: presence and condition of crash related traffic control devices at the time of the crash. Control devices include regulatory, warning, and construction signs. This excludes striping and officers or other persons directing traffic.
- `vz_pcf_description`: description of the California vehicle code primary collision factor violated.
- `dph_col_grp_description`: description of crash groupings.
- `party_number_ckey`: a unique identifier for each party involved in the crash. Used in combination with `case_id_pkey` to identify unique party.    
- `party_type`: type of party involved.
- `at_fault`: indicates if the party was at fault for the crash (Yes/No).
- `party_sex`: gender of the party involved.
- `party_age`: age of the party involved.
- `race`: racial classification of the party involved.
- `party_sobriety`: the state of sobriety of the party.    
- `party_safety_equip_1`: safety equipment used by the party (e.g. seat belt).
- `party_safety_equip_2`: additional safety equipment used by the party, if any.
- `finan_respons`: financial responsibility of the party.
- `party_number_killed`: number of fatalities for the party involved.
- `party_number_injured`: number of injuries for the party involved.    
- `move_pre_acc`: movement of the party before the collision.
- `vehicle_year`: year of the vehicle involved.
- `vehicle_make`: make/brand of the vehicle involved.
- `vehicle_age`: age of the vehicle involved in the crash, in years.
- `Current Police Districts`: police district in which the collision occurred.
- `Current Supervisor Districts`: supervisory district where the incident occurred.
    
Note: column descriptions from https://data.sfgov.org/Public-Safety/Traffic-Crashes-Resulting-in-Injury-Parties-Involv/8gtc-pjc6/about_data and https://tims.berkeley.edu/help/SWITRS.php.

## Python Initialization

First, we import pandas. Then, we read the data file (a csv) as a DataFrame, and store it in the variable `df`. The column `case_id_pkey` contains mixed data types (strings mixed with numbers). Using `dtype={'case_id_pkey': str}`, we specify we want to read it as a string.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/D-G-D/Python/main/Data/sf_traffic.csv', dtype={'case_id_pkey': str})

## Exploration
Add here all the code you run before answering the questions, in one or multiple code cells, and explain what you are doing.

# Questions

### Question 1. Are there any missing values in the data? If yes, what field is missing the most values?

Answer:

### Question 2. Are there any duplicate rows in the data? If yes, drop them.

Answer:

### Question 3. Are there any duplications of crash data?
Take the following list of columns `columns_list`. Are there any duplicated rows if we only look at those columns? Which % of the total they represent? Explain why this happens in the data.

In [ ]:
columns_list = ['case_id_pkey', 'tb_latitude', 'tb_longitude', 'collision_datetime',
       'officer_id', 'primary_rd', 'secondary_rd', 'distance', 'direction',
       'weather_1', 'weather_2', 'collision_severity', 'type_of_collision',
       'mviw', 'ped_action', 'road_surface', 'road_cond_1', 'road_cond_2',
       'lighting', 'control_device', 'vz_pcf_description',
       'dph_col_grp_description']

Answer:

### Question 4. Define a new column `intersection`.
The original data has a column `intersection` that isn't a Yes/No column as per the definition in the Statewide Integrated Traffic Records System (SWITRS, `https://tims.berkeley.edu/help/SWITRS.php`) codebook. Instead, it seems to define an intersection as any crash happening at most 20 feet from the intersection itself. However, there is one value (`Intersection Rear End <= 150ft` that would contradict this definition.

To start, let's take any row with `distance` smaller or equal to 20 ft as an intersection. Create a column named `intersection` that is True/False based on this definition. How many rows meet the criteria?

Answer:

### Question 5: Evaluate the impact of the "20 ft" choice.
Repeat question 4, but this time do it for all values of the distance threshold between 0 and 150 (20ft in Q4). Calculate the % of all rows that are interesctions. Plot the result (x-axis: distance threshold, y-axis: % of rows).

Answer:

### Question 6: What does the distribution of the party_number_injured column look like? When plotting it, is the default scale appropiate?

Answer:

### Question 7. Are the columns `vehicle_year`and `vehicle_age` associated? Justify your answer.

Answer:

### Question 8. Case data for injuries and lighting.
Filter the DataFrame to keep only the columns `case_id_pkey`, `lighting` and `party_number_injured`. Then, use `.groupby()` to calculate for each combination of `case_id_pkey` and `lighting` the sum of `party_number_injured`. What is the meaning of the resulting table?

Answer:

### Question 9: Investigate the impact of lighting to injuries.
First, use the code from the previous question to store the result in new variable `df_bycase`. Add `as_index=False` in the `.groupby()` to add all the values as columns instead of some as index. Then, keep the `lighting` and `party_number_injured` columns.

Use the new variable `df_bycase` to `groupby` again, this time calculating the average value of `party_number_injured` for each possible value of `lighting`. What is the meaning of the new table? What conclusions do you draw from the result?

Answer:

### Question 10. Extend your analysis to other parameters.
Leveraging the code from the previous question, repeat the analysis for 2 more columns other than `lighting`. If you can, re-define the code as a function that takes the column name as a parameter.

Answer: